# Hessian Matrix in Quantum Chemistry | Introduction

## 1. What is the Hessian in Quantum Chemistry?

The **Hessian matrix** in quantum chemistry is a second-derivative matrix that describes how the potential energy of a molecule changes with respect to atomic displacements. It is mathematically expressed as:

$$
H_{ij} = \frac{\partial^2 E}{\partial x_i \partial x_j}
$$

where:
- $ E $ is the total molecular potential energy,
- $ x_i $ and $ x_j $ are Cartesian coordinates of atoms.

Since energy is a function of atomic positions, the Hessian provides information about the **curvature of the potential energy surface (PES)**, which is crucial for understanding molecular stability, vibrational motions, and chemical reactivity.

### Importance of the Hessian Matrix
- **Characterization of stationary points:** The Hessian helps classify whether a molecular geometry corresponds to a **local minimum** (all eigenvalues positive) or a **transition state** (one eigenvalue negative).
- **Vibrational analysis:** The Hessian determines vibrational modes and frequencies, which are essential for **infrared (IR) and Raman spectroscopy**.
- **Molecular dynamics and reactions:** The Hessian plays a role in reaction pathway calculations, such as **intrinsic reaction coordinate (IRC) calculations**.


## 2. How is the Hessian Related to Vibrational Frequencies and Normal Modes?

The Hessian is directly linked to **vibrational frequencies** and **normal modes** because it contains the force constants governing atomic motions. The key relationships are:

### 2.1 Vibrational Motion and the Harmonic Approximation
In quantum chemistry, molecular vibrations are often described using the **harmonic approximation**, where the potential energy surface near equilibrium is approximated by a **quadratic function**:

$$
V(\mathbf{q}) = V_0 + \frac{1}{2} \sum_{i,j} H_{ij} q_i q_j
$$

where $ q_i $ and $ q_j $ are atomic displacements along Cartesian coordinates.

### 2.2 How Can We Calculate Vibrational Frequencies from the Hessian?

To compute **vibrational frequencies** from the Hessian matrix, follow these steps:

#### 2.2.1 Step 1: Compute the Hessian Matrix
- Perform a **geometry optimization** to locate a stationary point (minimum or transition state).
- Compute the **second derivatives** of energy with respect to Cartesian coordinates, forming the Hessian matrix.

$$
\mathbf{H}_{ij} = \frac{\partial^2 E}{\partial x_i \partial x_j}
$$

This matrix contains information about the molecular stiffness in different directions.

#### 2.2.2 Step 2: Convert the Hessian to Mass-Weighted Form
- The Hessian is converted to a **mass-weighted Hessian** by dividing each element by the square root of the corresponding atomic masses:

$$
\mathbf{H'}_{ij} = \frac{\mathbf{H}_{ij}}{\sqrt{m_i m_j}}
$$

where $ m_i $ and $ m_j $ are the masses of atoms corresponding to coordinates $ i $ and $ j $.

- The mass-weighted Hessian matrix is given by:

$$
\mathbf{H}' = \mathbf{M}^{-\frac{1}{2}} \mathbf{H} \mathbf{M}^{-\frac{1}{2}}
$$

where $ \mathbf{M} $ is the diagonal mass matrix:

$$
\mathbf{M} = \text{diag} ( m_1, m_1, m_1,\, m_2, m_2, m_2,\, \ldots,\, m_N, m_N, m_N ).
$$

#### 2.2.3 Step 3: Diagonalize the Mass-Weighted Hessian
- Solve the **eigenvalue problem** for the mass-weighted Hessian:

$$
\mathbf{H}' \mathbf{v} = \lambda \mathbf{v}
$$

- The eigenvalues $ \lambda_i $ correspond to the vibrational force constants.
- The eigenvectors $ \mathbf{v}_i $ represent the normal mode displacements.

#### 2.2.4 Step 4: Compute Vibrational Frequencies
- Convert eigenvalues to frequencies:

$$
\nu_i = \sqrt{\lambda_i}
$$

- Frequencies are often converted to **wavenumbers** ($ \tilde{\nu} $, in $ cm^{-1} $):

$$
\tilde{\nu}_i = \frac{\nu_i}{2\pi c}
$$

where $ c $ is the speed of light in cm/s.

#### 2.2.5 Step 5: Assign Vibrational Modes
- Analyze the normal modes to determine their nature:
  - **Real, positive frequencies** correspond to **stable vibrations**.
  - **One imaginary frequency ($\nu_i^2 < 0$)** indicates a **transition state**.


## 3. Summary

| Step | Description |
|------|------------|
| **1. Compute Hessian** | Obtain second derivatives of energy with respect to atomic displacements. |
| **2. Convert to Mass-Weighted Form** | Divide each element by $ \sqrt{m_i m_j} $ to account for atomic masses. |
| **3. Diagonalize the Matrix** | Solve the eigenvalue problem to obtain normal modes. |
| **4. Compute Frequencies** | Convert eigenvalues to vibrational frequencies using $ \nu =\sqrt{\lambda} $. |
| **5. Assign Vibrational Modes** | Identify stretching, bending, and other vibrational motions. |

# Analyse Hessian using Python

## 1. Runing ORCA Optimizatin and Frequency Calculation for $\mathbf{H_2O}$

```
! PAL2
! Opt Freq B3LYP Def2-TZVP Def2/J DefGrid2
! TightSCF
! LargePrint

%MaxCore 1000

* xyz 0 1
O    0.0000   -0.7920    0.7920
H    0.0000    0.0000    0.0000
H    0.0626   -0.4973   -0.4973
*
```

## 2. Importing libraries

In [ ]:
import re
import numpy as np
import pandas as pd
from io import StringIO

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

## 3. Defining the conversion factors

In [2]:
bohr2m = 0.529177249e-10
bohr2ang = 0.529177249
au2joule = 4.35974434e-18
au2ev = 2.72114e1
cmInv2au = 4.55634e-6
amu2au = 1./5.4857990943e-4

## 4. Defining the functions and do the calculations

### 4.1 Capturing basic information about the chemical structure

In [45]:
def analyse_basic(hessian_path):
    
    with open(hessian_path, 'r') as f:
        hess = f.read()

    regex = r'\$atoms\s+\d+\s+((?:\s+[a-zA-Z]+\s+\d+.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+.+)+)'

    matches = re.search(regex, hess).group()
    matches = matches.split('\n')[2:]
    matches = '\n'.join(matches)

    df = pd.read_csv(StringIO(matches), sep=r'\s+', header=None)

    df.columns = ['Symbol', 'Mass', 'X', 'Y', 'Z']
    
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * bohr2ang

    df['Mass'] = df['Mass'] * amu2au
    
    return df
    # return matches

In [46]:
df_basic = analyse_basic(hessian_path='orca_job/h2o.hess')

print('Basic Information')
print(df_basic)

Basic Information
  Symbol          Mass         X         Y         Z
0      O  29164.392871  0.014997 -0.654708  0.416430
1      H   1837.471593 -0.043060  0.305760  0.378384
2      H   1837.471593  0.090663 -0.940352 -0.500114


### 4.2 Capturing the hessian matrix ($\mathbf{H}$)

In [5]:
def analyse_hessian(hessian_path, atom_number):

    dof = 3 * atom_number
    
    with open(hessian_path, 'r') as f:
        hess = f.read()
    
    regex = r'(?ms)\$hessian\s\d+(.*)\$vibrational_frequencies'
    
    hess_lines = re.search(regex, hess).group().split('\n')[2:]

    hess_lines_2 = list(zip(*(iter(hess_lines),) * (dof+1)))

    hess_data = {str(i):[] for i in range(dof)}

    for i in hess_lines_2:
        col_nums = i[0].strip().split()
        
        for j in range(1,dof+1):        
            for k in range(len(col_nums)):
                hess_data[col_nums[k]].append(float(i[j].strip().split()[k+1]))
            
    hessian_mat = pd.DataFrame(hess_data).to_numpy()

    return hessian_mat            

In [31]:
hessian = analyse_hessian(hessian_path='orca_job/h2o.hess', atom_number=3)

print('Hessian (Hartree/Bohr^2)')
print(pd.DataFrame(hessian))

Hessian (Hartree/Bohr^2)
          0         1         2         3         4         5         6         7         8
0  0.005303 -0.044138 -0.039168 -0.002058  0.032593 -0.000468 -0.003244  0.011545  0.039635
1 -0.044129  0.592793  0.107969  0.028873 -0.510132  0.051611  0.015256 -0.082658 -0.159575
2 -0.039095  0.108016  0.514413  0.003286 -0.007918 -0.044067  0.035809 -0.100099 -0.470341
3 -0.002050  0.028851  0.003274  0.002286 -0.032985 -0.002785 -0.000236  0.004135 -0.000489
4  0.032591 -0.510112 -0.007914 -0.032985  0.526504 -0.002207  0.000394 -0.016393  0.010118
5 -0.000479  0.051582 -0.044076 -0.002785 -0.002207  0.046513  0.003264 -0.049375 -0.002434
6 -0.003269  0.015243  0.035893 -0.000236  0.000394  0.003264  0.003505 -0.015638 -0.039157
7  0.011503 -0.082647 -0.100067  0.004135 -0.016394 -0.049375 -0.015637  0.099041  0.149440
8  0.039644 -0.159557 -0.470334 -0.000489  0.010118 -0.002434 -0.039156  0.149439  0.472760


### 4.3 Building the mass matrix ($\mathbf{M}$)

In [7]:
def build_mass_matrix(masses):
    N = len(masses)
    size = 3 * N
    M = np.zeros((size, size))
    for i in range(N):
        M[3*i, 3*i] = masses[i]
        M[3*i+1, 3*i+1] = masses[i]
        M[3*i+2, 3*i+2] = masses[i]
    return M

In [30]:
M = build_mass_matrix(df_basic['Mass'].values)

print('Mass matrix (au)')
print(pd.DataFrame(M))

Mass matrix (au)
              0             1             2            3            4            5            6            7            8
0  29164.392871      0.000000      0.000000     0.000000     0.000000     0.000000     0.000000     0.000000     0.000000
1      0.000000  29164.392871      0.000000     0.000000     0.000000     0.000000     0.000000     0.000000     0.000000
2      0.000000      0.000000  29164.392871     0.000000     0.000000     0.000000     0.000000     0.000000     0.000000
3      0.000000      0.000000      0.000000  1837.471593     0.000000     0.000000     0.000000     0.000000     0.000000
4      0.000000      0.000000      0.000000     0.000000  1837.471593     0.000000     0.000000     0.000000     0.000000
5      0.000000      0.000000      0.000000     0.000000     0.000000  1837.471593     0.000000     0.000000     0.000000
6      0.000000      0.000000      0.000000     0.000000     0.000000     0.000000  1837.471593     0.000000     0.000000
7      

### 4.4 Transforming the Hessian to Mass-weighted Hessian ($\mathbf{H'}$)

In [14]:
def mass_weighted_hessian(H, M):
    M_inv_sqrt = np.diag(1.0 / np.sqrt(np.diag(M)))
    H_prime = M_inv_sqrt @ H @ M_inv_sqrt
    return H_prime

In [47]:
H_prime = mass_weighted_hessian(hessian, M)

print('Mass-weighted Hessian or H_prime')
print(pd.DataFrame(H_prime))

Mass-weighted Hessian or H_prime
              0         1             2             3             4             5             6         7             8
0  1.818463e-07 -0.000002 -1.343000e-06 -2.810694e-07  4.452346e-06 -6.392549e-08 -4.431977e-07  0.000002  5.414241e-06
1 -1.513103e-06  0.000020  3.702067e-06  3.944148e-06 -6.968601e-05  7.050313e-06  2.084073e-06 -0.000011 -2.179854e-05
2 -1.340515e-06  0.000004  1.763838e-05  4.489110e-07 -1.081573e-06 -6.019680e-06  4.891685e-06 -0.000014 -6.425047e-05
3 -2.800878e-07  0.000004  4.472490e-07  1.244066e-06 -1.795148e-05 -1.515501e-06 -1.286570e-07  0.000002 -2.659078e-07
4  4.452028e-06 -0.000070 -1.081109e-06 -1.795148e-05  2.865370e-04 -1.200851e-06  2.145643e-07 -0.000009  5.506282e-06
5 -6.547575e-08  0.000007 -6.020970e-06 -1.515483e-06 -1.200850e-06  2.531384e-05  1.776303e-06 -0.000027 -1.324687e-06
6 -4.465710e-07  0.000002  4.903088e-06 -1.286269e-07  2.146140e-07  1.776354e-06  1.907400e-06 -0.000009 -2.131001e-05
7  1.57

### 4.5 Diagonalizing the $\mathbf{H'}$

In [16]:
def diagonalize_hessian(H_prime):
    eigenvals, eigenvects = np.linalg.eigh(H_prime)
    return eigenvals, eigenvects

In [48]:
evals, evecs = diagonalize_hessian(H_prime)

print('Eigenvalues')
print(pd.DataFrame(evals))

print('\n')

print('Eigenvectors')
print(pd.DataFrame(evecs))


Eigenvalues
              0
0 -3.076483e-09
1  1.483582e-09
2  1.886369e-09
3  1.989806e-08
4  2.318461e-08
5  3.271871e-08
6  5.424279e-05
7  2.966778e-04
8  3.133407e-04


Eigenvectors
          0         1         2         3         4         5         6         7         8
0 -0.655774 -0.384841 -0.555452 -0.275332  0.038279  0.188336 -0.004089  0.002914 -0.023716
1 -0.335000  0.854803 -0.204155 -0.013463 -0.169799  0.015654 -0.157955  0.110944  0.221112
2  0.595819  0.059500 -0.727250 -0.046818 -0.102753  0.030671  0.223407 -0.157191  0.155742
3 -0.146367 -0.106050 -0.158453  0.943385 -0.068503  0.204841  0.043714  0.044720  0.041046
4 -0.083223  0.214255 -0.052421  0.061495 -0.018977  0.012002 -0.016547 -0.691808 -0.679163
5  0.143501  0.021570 -0.183194  0.081471  0.690310 -0.028801 -0.678453 -0.019076  0.026902
6 -0.149661 -0.105350 -0.152163  0.141152 -0.092933 -0.952896 -0.027371 -0.056337  0.053440
7 -0.089995  0.220915 -0.051526  0.038057  0.638833 -0.103361  0.645834  0.24

### 3.6 Computing the frequencies

In [26]:
def compute_frequencies(eigenvals):
    threshold = 1e-6
    positive_eigs = np.array([ev if ev > threshold else 0.0 for ev in eigenvals])
    freqs = np.sqrt(positive_eigs) 
    freqs_cmInv = freqs / cmInv2au
    return freqs_cmInv

In [27]:
freqs = compute_frequencies(evals) 

print('Frequencies (cm-1)')
print(pd.DataFrame(freqs))

Frequencies (cm-1)
             0
0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6  1616.422512
7  3780.301208
8  3885.011487


# Exercise

Reproduce the normal modes from the ```h2o.hess``` file.

! Hint: You should use the eigenvectors calculated in the previous section 4.5 of **Analyse Hessian using Python**.